In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt5

In [2]:
prefix = "/home/pleroy/DATA/PoSAR-MC_PIMA_TEST-2/2017_09_22_"
data_dir = prefix + "11_43_45"

In [41]:
blocksPerFile = 6
samplesPerRamp = 6000
rampsPerFile = 1500
samplesPerFile = samplesPerRamp * rampsPerFile

numberOfFiles = 15
firstFile = 6690 # 6684
lastFile = firstFile + numberOfFiles * blocksPerFile
nbFilesToCheck = numberOfFiles
nbFilesToRebuild = numberOfFiles

### Read the data

In [4]:
def readFile( filename, timeSerie_A, timeSerie_B ):
    fd = open(filename,'rb')
    
    dum = np.fromfile(fd, dtype = np.int16)

    timeSerie_A[:] = dum[ 0 : 2 * samplesPerFile : 2 ]
    timeSerie_B[:] = dum[ 1 : 2 * samplesPerFile : 2 ]
    
    fd.close()

In [5]:
timeSerie_A = np.zeros( samplesPerFile )
timeSerie_B = np.zeros( samplesPerFile )

### Check pulse duration in B channel data

In [42]:
threshold = 6000

In [43]:
for k in range(nbFilesToCheck):
    nb = str(int(k * blocksPerFile + firstFile))
    readFile( data_dir + "/record" + nb +".bin", timeSerie_A, timeSerie_B )
    pulseDuration = np.where(timeSerie_B > threshold)[0].shape[0]
    print("file " + str(nb) + ", pulseDuration =  " + str(pulseDuration))

file 6690, pulseDuration =  1000
file 6696, pulseDuration =  1000
file 6702, pulseDuration =  1000
file 6708, pulseDuration =  1000
file 6714, pulseDuration =  1000
file 6720, pulseDuration =  1000
file 6726, pulseDuration =  1000
file 6732, pulseDuration =  1000
file 6738, pulseDuration =  1000
file 6744, pulseDuration =  1000
file 6750, pulseDuration =  1000
file 6756, pulseDuration =  1000
file 6762, pulseDuration =  1000
file 6768, pulseDuration =  1000
file 6774, pulseDuration =  1000


### Check the triggers in the B channel data

In [44]:
for k in range(nbFilesToCheck):
    nb = str(int(k * blocksPerFile + firstFile))
    readFile( data_dir + "/record" + nb +".bin", timeSerie_A, timeSerie_B )
    triggers = np.where( np.diff(timeSerie_B) > threshold )[0]
    print("file " + str(nb) + ", threshold = " + str(triggers))

file 6690, threshold = [8990808]
file 6696, threshold = [8993680]
file 6702, threshold = [8993551]
file 6708, threshold = [8993442]
file 6714, threshold = [8993275]
file 6720, threshold = [8993160]
file 6726, threshold = [8993080]
file 6732, threshold = [8992970]
file 6738, threshold = [8992865]
file 6744, threshold = [8992774]
file 6750, threshold = [8992559]
file 6756, threshold = [8992449]
file 6762, threshold = [8992350]
file 6768, threshold = [8992211]
file 6774, threshold = [8992113]


# Rebuild the data files

In [45]:
timeSerie_C = np.zeros( samplesPerFile )
timeSerie_D = np.zeros( samplesPerFile )

In [46]:
rebuilt_A = np.zeros((nbFilesToRebuild, samplesPerFile))
rebuilt_B = np.zeros((nbFilesToRebuild, samplesPerFile))
for k in range(nbFilesToRebuild):
    nb        = str( int( k     * blocksPerFile + firstFile) )
    nb_plus_1 = str( int( (k+1) * blocksPerFile + firstFile) )
    readFile( data_dir + "/record" + nb        +".bin", timeSerie_A, timeSerie_B )
    readFile( data_dir + "/record" + nb_plus_1 +".bin", timeSerie_C, timeSerie_D )
    trigs = np.where( np.diff(timeSerie_B) > threshold )[0]
    trig = trigs[0] + 1

    rebuilt_B[ k, 0 : samplesPerFile - trig ] = timeSerie_B[ trig : ]
    rebuilt_B[ k, samplesPerFile - trig : ] = timeSerie_D[ 0 : trig ]
    
    rebuilt_A[ k, 0 : samplesPerFile - trig ] = timeSerie_A[ trig : ]
    rebuilt_A[ k, samplesPerFile - trig : ] = timeSerie_C[ 0 : trig ]
    
    print("file " + str(nb) + ", trigs = " + str(trigs))

file 6690, trigs = [8990808]
file 6696, trigs = [8993680]
file 6702, trigs = [8993551]
file 6708, trigs = [8993442]
file 6714, trigs = [8993275]
file 6720, trigs = [8993160]
file 6726, trigs = [8993080]
file 6732, trigs = [8992970]
file 6738, trigs = [8992865]
file 6744, trigs = [8992774]
file 6750, trigs = [8992559]
file 6756, trigs = [8992449]
file 6762, trigs = [8992350]
file 6768, trigs = [8992211]
file 6774, trigs = [8992113]


In [27]:
plt.plot(rebuilt_B[1,:])

In [47]:
A_reshaped = rebuilt_A.reshape(nbFilesToRebuild * rampsPerFile, samplesPerRamp)
B_reshaped = rebuilt_B.reshape(nbFilesToRebuild * rampsPerFile, samplesPerRamp)

In [48]:
coupling_A = np.average(A_reshaped, 0)
A_reshaped -= coupling_A

In [16]:
for k in range(nbFilesToRebuild):
    plt.plot(A_reshaped[k,0:6000])

In [49]:
ifft_a = np.fft.ifft( (A_reshaped[::,0:3000]), axis = 1 )

In [56]:
azMax = 0
azMin = ifft_a.shape[1]
rangeMin = 500
rangeMax = 2000
plt.imshow( 20 * np.log10( np.abs( ifft_a[:,500:2000] ) ),
           aspect='auto', extent=(rangeMin, rangeMax, azMin, azMax) )
plt.title("Track 1 / Flight over 2\nFiles {} to {}".format(firstFile, lastFile) )

In [54]:
np.save(data_dir + '/samplesA_track3_flightover1.npy', A_reshaped)

# Annexe

In [ ]:
rd_a = np.fft.fft( ifft_a, axis = 0 )

In [ ]:
plt.matshow( 20 * np.log10( np.abs( rd_a[:, 0:500] ) ), aspect='auto' )

Track 1 Flight over 2, average altitude = 227.281, velocity = 44.250

In [53]:
np.save('A_reshaped_tr3_fo1.npy', A_reshaped)

In [ ]:
plt.figure()
plt.plot(A_reshaped[0,0:3000])
plt.plot(A_reshaped[0,3000:6000])
plt.plot(A_reshaped[1,0:3000])
plt.plot(A_reshaped[1,3000:6000])

In [ ]:
plt.figure()
ax1 = plt.subplot(211)
plt.plot( np.real( np.fft.fft( A_reshaped[0,0:3000] ) ) )
plt.plot( np.real( np.conjugate( np.fft.fft( A_reshaped[0,3000:6000] ) ) ) )
plt.subplot(212, sharex = ax1)
plt.plot( np.imag( np.fft.fft( A_reshaped[0,0:3000] ) ) )
plt.plot( np.imag( np.conjugate( np.fft.fft( A_reshaped[0,3000:6000] ) ) ) )

In [ ]:
plt.plot( np.fft.ifft( np.conjugate( np.fft.fft( A_reshaped[0,3000:6000] ) ) ) )

In [ ]:
alpha = 150e6 / 300e-6
c = 3e8
fs = 10e6

In [ ]:
fs * c / ( 2 * alpha )

In [ ]:
3000*2/c*0.5e12

In [ ]:
np.fft.fft( A_reshaped[0,0:3000] )